In [1]:
import os
import sys
import math
import logging
from pathlib import Path

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from snowflake.core import Root
from snowflake import cortex
from snowflake.snowpark import Session

In [3]:
session = Session.builder.config('connection_name', 'default').create()
root = Root(session)

INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.8.1, Python Version: 3.10.9, Platform: macOS-14.4.1-arm64-arm-64bit


INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.snowpark.session:Snowpark Session information: 
"version" : 1.15.0,
"python.version" : 3.10.9,
"python.connector.version" : 3.8.1,
"python.connector.session.id" : 26798426968733078,
"os.name" : Darwin

INFO:snowflake.core.session._generated.api.session_api:Going to use client-REST for this resource


In [20]:
session.sql("SHOW WAREHOUSES").collect()

INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0


[Row(name='COMPUTE_WH', state='SUSPENDED', type='STANDARD', size='X-Small', min_cluster_count=1, max_cluster_count=1, started_clusters=0, running=0, queued=0, is_default='Y', is_current='Y', auto_suspend=600, auto_resume='true', available='', provisioning='', quiescing='', other='', created_on=datetime.datetime(2024, 5, 3, 7, 53, 54, 800000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), resumed_on=datetime.datetime(2024, 5, 4, 14, 46, 20, 368000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), updated_on=datetime.datetime(2024, 5, 4, 14, 46, 20, 368000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), owner='ACCOUNTADMIN', comment='', enable_query_acceleration='false', query_acceleration_max_scale_factor=8, resource_monitor='null', actives=0, pendings=0, failed=0, suspended=1, uuid='1597310724', scaling_policy='STANDARD', budget=None, owner_role_type='ROLE')]

In [43]:
def create_wh(name:str, **kwargs) -> Warehouse:
    
    root = Root(Session.builder.getOrCreate())
    wh_cfg = Warehouse(name=name, **kwargs)
    return root.warehouses[name].create_or_update(wh_cfg)

In [49]:
def create_wh(name: str, **kwargs) -> WarehouseResource:
    """Create a warehouse in the current Snowflake session"""
    root = Root(Session.builder.getOrCreate())
    wh_cfg = Warehouse(name=name, **kwargs)
    # This should work according to the Snowflake documentation but it's not working in the current version.
    # Raises: NotImplementedError: create_or_update is not yet supported for warehouse. Updating warehouse objects\
    # is not supported yet; use create() for creating a warehouse.
    # wh = root.warehouses[name].create_or_update(wh_cfg)
    wh_collection = WarehouseCollection(root)
    return wh_collection.create(wh_cfg)

In [54]:
def get_wh(name: str) -> WarehouseResource:
    """Retrieve a warehouse in the current Snowflake session"""
    root = Root(Session.builder.getOrCreate())
    wh_collection = WarehouseCollection(root)
    return wh_collection[name]

In [55]:
wh = get_wh("snow_instructor")

INFO:snowflake.core.session._generated.api.session_api:Going to use client-REST for this resource


In [56]:
wh.name

'snow_instructor'

In [48]:
from snowflake.core.warehouse import Warehouse
warehouse_parameters = Warehouse(
     name="your_warehouse_name",
     warehouse_size="SMALL",
     auto_suspend=500,
)
# Use warehouse collection to create a reference to warehouse resource in Snowflake server.
root.warehouses["your_warehouse_name"].create_or_update(warehouse_parameters)

NotImplementedError: create_or_update is not yet supported for warehouse. Updating warehouse objects is not supported yet; use create() for creating a warehouse.

In [46]:
create_wh("compute_wh2")

INFO:snowflake.core.session._generated.api.session_api:Going to use client-REST for this resource


INFO:snowflake.core.warehouse._generated.api.warehouse_api:Going to use client-REST for this resource


NotImplementedError: create_or_update is not yet supported for warehouse. Updating warehouse objects is not supported yet; use create() for creating a warehouse.

In [ ]:
root.warehouses.

In [38]:
from snowflake.core.warehouse import WarehouseCollection, WarehouseResource, Warehouse

wh_collection = WarehouseCollection(root)
wh = root.warehouses['snowflake_instructor_wh']
wh = WarehouseResource.create_or_update(wh)

TypeError: WarehouseResource.create_or_update() missing 1 required positional argument: 'warehouse'

In [35]:
wh_resource.create_or_update()

INFO:snowflake.core.warehouse._generated.api.warehouse_api:Going to use client-REST for this resource


TypeError: WarehouseResource.create_or_update() missing 1 required positional argument: 'warehouse'

In [16]:
list(wh_collection.keys())

[]

In [4]:
with open('../page.md', 'r') as f:
    text = f.read()

In [5]:
prompt = f"""
I am your student and I am learning how to use Snowflake with Python.
You are a nice teacher that helps me, your student, to learn how to use Snowflake.

You output a question, 4 answers, the correct answer, and an explanation in HTML format.
Here is an example:

<question>What is the purpose of the `Session.builder.configs` method in creating a session for Snowpark Python?</question>
<answers>
<ol type="1">
<li> To establish a connection to the Snowflake database </li>
<li> To create a dictionary containing the names and values of the connection parameters </li>
<li> To return a builder object that has the connection parameters </li>
<li> To authenticate the user using the Snowflake Connector for Python  </li>
</ol>
</answers>
<correct>To return a builder object that has the connection parameters</correct>
<explanation>The `Session.builder.configs` method is used to create a builder object that has the connection parameters. This builder object can then be used to create a session object that can be used to execute queries against the Snowflake database. The method takes a dictionary of connection parameters as an argument, which contains the names and values of the parameters that are needed to connect to the database. The method returns a builder object that has the connection parameters, which can then be used to create a session object.</explanation>

Take now the following page from the Snowflake documentation and formulate a question for me:
{text}
"""

In [6]:
prompt = f"""
"Based on the following excerpt from the Snowflake documentation, generate a multiple-choice question that tests understanding of the key concept discussed. Include four answer choices and indicate the correct answer. Format the output in HTML.
{text}
"""

In [7]:
print(prompt)


"Based on the following excerpt from the Snowflake documentation, generate a multiple-choice question that tests understanding of the key concept discussed. Include four answer choices and indicate the correct answer. Format the output in HTML.
[Developer](/en/developer)[Kafka and Spark Connectors](/en/user-guide/connectors)[Kafka Connector](/en/user-guide/kafka-connector)Overview

Overview of the Kafka connector[¶](#overview-of-the-kafka-connector "Link to this heading")


This topic provides an overview of the Apache Kafka and the Snowflake Connector for Kafka.



Note


The Kafka connector is subject to the [Connector Terms](https://www.snowflake.com/legal/snowflake-connector-terms).




Introduction to Apache Kafka[¶](#introduction-to-apache-kafka "Link to this heading")
-------------------------------------------------------------------------------------


Apache Kafka software uses a publish and subscribe model to write and read streams of records, similar to a message queue or 

In [8]:
model = 'snowflake-arctic'
# model = 'llama2-70b-chat'
answer = cortex.Complete(model=model, prompt=prompt, session=session)

INFO:snowflake.connector.cursor:Number of results in first chunk: 0


SnowparkSQLException: (1300) (1304): 01b41ac4-0002-65f7-005f-35070001445e: 100351 (P0000): 01b41ac4-0002-65f7-005f-35070001445e: Request failed for external function COMPLETE$V2 with remote service error: 400 '"max tokens of 4096 exceeded"
'; requests batch-id: 01b41ac4-0002-65f7-005f-35070001445e:1:1:0:0; request batch size: 1 rows; request retries: 0; response time (last retry): 44.53ms

In [9]:
print(answer)

NameError: name 'answer' is not defined

In [15]:
print(answer)

 Which of the following is the correct way to create a session for Snowpark Python?

A. `from snowflake.snowpark import Session`
B. `Session.builder.configs(connection_parameters).create()`
C. `new_session = Session.builder.configs(connection_parameters).create()`
D. `new_session = Session.builder.create(connection_parameters)`

Correct answer: C. `new_session = Session.builder.configs(connection_parameters).create()`
